#	Cross Matching Catalogues using kd-Trees data structures with AstroPy in O(n*log(n))

Cross Matching AT20GBSS and SuperCOSMOS Sky Surveys Catalogues using kd-Trees data structures with AstroPy in O(n*log(n))

Here, we have a sample dataset from <b>AT20GBSS Catalogue</b> named <b>bss.dat</b> and <b>SuperCOSMOS Sky Survey Catalogues</b> named <b>super.csv</b>. 

The crossmatch function crossmatches two catalogues within a maximum distance. It should return a list of matches and non-matches for the first catalogue against the second.

The list of matches contains tuples of the first and second catalogue object IDs and their distance. The list of non-matches contains the unmatched object IDs from the first catalogue only. Both lists should be ordered by the first catalogue's IDs.

The BSS and SuperCOSMOS catalogues will be given as input arguments. The maximum distance is given in decimal degrees.

First we try the traditional method of cross matching two catalogues - <b>for every object in BSS, we calculate distance to every object in SuperCOSMOS.</b>

In [1]:
import numpy as np

def hms2dec(h, m, s):
  return 15*(h + m/60 + s/3600)

def dms2dec(d, m, s):
  if (d >= 0):
    return d + m/60 + s/3600
  else:
    return d - m/60 - s/3600 

def import_bss():
  file = 'bss.dat'
  lines = np.loadtxt(file, usecols=range(1, 7))
  
  count=1
  result = [ ]
  for line in lines:
    result.append((count, hms2dec(line[0], line[1], line[2]), dms2dec(line[3], line[4]
, line[5])))
    count += 1
  return result

def import_super():
  file = 'super.csv'
  lines = np.loadtxt(file, delimiter=',', skiprows=1,usecols=[0,1])
  result = []
  count = 1
  for line in lines:
    result.append((count, line[0], line[1]))
    count += 1
  return result

def angular_dist(a1, d1, a2, d2):
  a1 = np.radians(a1)
  d1 = np.radians(d1)
  a2 = np.radians(a2)
  d2 = np.radians(d2)

  p1 = np.sin(abs(d1-d2)/2)**2
  p2 = np.cos(d1)*np.cos(d2)*np.sin(abs(a1-a2)/2)**2
  p3 = 2*np.arcsin(np.sqrt(p1+p2))
  return np.degrees(p3)

def crossmatch(cat1, cat2, max_dist):
  matches = []
  nomatches = []
  
  for cat1line in cat1:
    found = False
    best_match = (0,0,max_dist+1)
    
    for cat2line in cat2:
      dist = angular_dist(cat1line[1], cat1line[2], cat2line[1], cat2line[2])
      if(dist <= best_match[2]):
        best_match = (cat1line[0], cat2line[0], dist)
    
    if(best_match[2] <= max_dist):
      matches.append( best_match )
    else:
      nomatches.append( cat1line[0] )
  
  return (matches, nomatches)

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  bss_cat = import_bss()
  super_cat = import_super()

  # First example in the question
  max_dist = 40/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  
  print(matches[:3])
  print(no_matches[:3])

  # Second example in the question
  max_dist = 5/3600
  matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
  print(matches)
  print(matches[:3])
  print(no_matches[:3])
  print("\n # of no Matches  :", len(no_matches))



[(1, 2, 0.00010988610938710059), (2, 4, 0.00076498459672424935), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
[(1, 2, 0.00010988610938710059), (2, 4, 0.00076498459672424935), (3, 5, 0.00020863352870707666), (4, 6, 0.00012867299967083928), (7, 11, 7.6662350900110081e-05), (8, 14, 0.00020833046101750643), (9, 15, 0.00017740150260883261), (10, 17, 0.00016079604439502975), (12, 18, 0.00034939446067522219), (13, 22, 0.00012170541597764247), (15, 25, 0.00021977416884437018), (16, 26, 0.00013282868901392909), (17, 27, 3.1815304278426531e-05), (18, 29, 0.00019843774684364136), (19, 30, 0.00011585178516956528), (20, 31, 0.00026271369223261241), (21, 32, 0.00039932330328371096), (22, 33, 7.3653557722555701e-05), (23, 34, 0.00022280948960772594), (24, 35, 0.00015481960583649628), (25, 37, 0.0003688060288937162), (26, 39, 0.00037503127127783077), (27, 41, 0.0001183449200434592), (28, 42, 0.00012217104062974945), (30, 43, 0.00055730322654573138), (31, 44, 0.00018231015875520086), (32, 46, 0.000232197

<h3>Algorithm clearly needs optimization</h3>
The way we've implemented our crossmatcher means that for every object in BSS, we need to calculate distance to every object in SuperCOSMOS. Even our small crossmatching task requires <b>160 × 500 = 80,000 distance calculations.</b>

With each distance calculation taking a few microseconds, it quickly adds up to seconds or minutes.

Seconds may not seem like much but remember that the full SuperCOSMOS catalogue has 126 million objects, over 250,000 times larger than the truncated version we gave you to work with.

Then, imagine you were trying to crossmatch a catalogue other than AT20G BSS, with a size comparable to SuperCOSMOS. A crossmatching operation like that might take months or years.

We clearly need to be smarter about our choice of algorithm.


<h3>1. Microoptimization</h3>
In the crossmatcher we developed abovey, it was necessary to convert the RA and declination from degrees to radians so that the trigonometric functions could work with them.

If this conversion occurred in the distance calculation function, then the same coordinates would be converted many times during the crossmatching operation.

In the next problem, we'll modify the crossmatcher algorithm so that the conversion occurs only once, before any distance calculations. This should save only a small amount of time, but it all adds up in the end.

Since our focus from now on will be improving our algorithm, we'll be using randomly generated catalogues instead of SuperCOSMOS and the AT20G BSS. This lets us see if our changes are improving our algorithm's efficiency in general, instead of just finding something that works for two specific catalogues.

In [2]:
import numpy as np
import statistics
import time

def hms2dec(h, m, s):
  return 15*(h + m/60 + s/3600)

def dms2dec(d, m, s):
  if (d >= 0):
    return d + m/60 + s/3600
  else:
    return d - m/60 - s/3600

def angular_dist(a1, d1, a2, d2):
  p1 = np.sin(abs(d1-d2)/2)**2
  p2 = np.cos(d1)*np.cos(d2)*np.sin(abs(a1-a2)/2)**2
  p3 = 2*np.arcsin(np.sqrt(p1+p2))
  return np.degrees(p3)

def crossmatch(cat1, cat2, max_dist):
  matches = []
  nomatches = []
  cat1id, cat2id = 0, 0
  
  start = time.perf_counter()
  
  cat1 = cat1.astype(float)
  cat2 = cat2.astype(float)

  for cat2line in cat2:
    cat2line[0] = np.radians(cat2line[0])
    cat2line[1] = np.radians(cat2line[1])
  
  for cat1line in cat1:
    cat1line[0] = np.radians(cat1line[0])
    cat1line[1] = np.radians(cat1line[1])
    found = False
    best_match = (0,0,max_dist+1)

    cat2id = 0
    for cat2line in cat2:
      
      dist = angular_dist(cat1line[0], cat1line[1], cat2line[0], cat2line[1])
      if(dist <= best_match[2]):
        best_match = (cat1id, cat2id, dist)
      cat2id += 1

    if(best_match[2] <= max_dist):
      matches.append(best_match)
    else:
      nomatches.append(cat1id)
      
    cat1id += 1
    
  return (matches, nomatches, time.perf_counter() - start)


# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('\n Time taken:', time_taken)

matches: [(0, 0, 2.0000000000000027), (2, 2, 1.7420109046547023)]
unmatched: [1]
time taken: 0.00018292456535040467
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

 Time taken: 0.0025630569049344652


<h3>2. Vectorization</h3>
NumPy has highly-optimised C and Fortran code specifically for numerical calculations.
The maths functions we used last time (np.sqrt, np.sin, np.cos and np.arcsin) apply to any NumPy number or array. This means we can speed up the part of crossmatch that searches through the second catalogue

In [3]:
import numpy as np
import statistics
import time

def hms2dec(h, m, s):
  return 15*(h + m/60 + s/3600)

def dms2dec(d, m, s):
  if (d >= 0):
    return d + m/60 + s/3600
  else:
    return d - m/60 - s/3600

def angular_dist(a1, d1, a2, d2):
  p1 = np.square(np.sin(np.absolute(d1-d2)/2))
  p2 = np.cos(d1)*np.cos(d2)*np.square(np.sin(np.absolute(a1-a2)/2))
  return 2*np.arcsin(np.sqrt(p1+p2))

def crossmatch(cat1, cat2, max_dist):
  matches, nomatches = [], []
  cat1id, cat2id = 0, 0

  start = time.perf_counter()

  # convert everything to radians
  cat1 = np.radians(cat1)
  cat2 = np.radians(cat2)
  max_dist = np.radians(max_dist)
  
  ra2s = cat2[:,0]
  dec2s = cat2[:,1]
  
  for cat1line in cat1:
    dists = angular_dist(cat1line[0], cat1line[1], ra2s, dec2s) 
    min_dist = np.min(dists)
    if(min_dist <= max_dist):
      matches.append((cat1id,np.argmin(dists),min_dist))
    else:
      nomatches.append(cat1id)
    
    cat1id += 1
    
  return (matches, nomatches, time.perf_counter() - start)

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  ra1, dec1 = np.radians([180, 30])
  cat2 = [[180, 32], [55, 10], [302, -44]]
  cat2 = np.radians(cat2)
  ra2s, dec2s = cat2[:,0], cat2[:,1]
  dists = angular_dist(ra1, dec1, ra2s, dec2s)
  print(np.degrees(dists))

  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('\n Time taken:', time_taken)

[   2.          113.72587199  132.64478705]
matches: [(0, 0, 0.03490658503988664), (2, 2, 0.03040382589186957)]
unmatched: [1]
time taken: 0.00016934105802239585
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

 Time taken: 0.0003338524245504043


<h3>3. Reducing Redundancies</h3>
Another optimisation we can make is to ignore objects in the second catalogue with a declination far from the first catalogue object currently being matched. 

The easiest way of doing this is:
    Loop through the second catalogue objects in order of declination, rather than ID;
    Stop when the declination of the second catalogue object exceeds the target declination by the maximum radius.

So, for example, say we have a first catalogue object (the target) with a declination of  and we’ve set our maximum match radius to . 
The new algorithm will only loop over second catalogue objects with declinations between -90 (the start) and (ϴ+r)  degrees before it breaks out of the loop.

If we have evenly distributed objects, this lets crossmatch avoid distance calculations to approximately half the second catalogue objects (on average).

In [4]:
import numpy as np
import statistics
import time

def hms2dec(h, m, s):
  return 15*(h + m/60 + s/3600)

def dms2dec(d, m, s):
  if (d >= 0):
    return d + m/60 + s/3600
  else:
    return d - m/60 - s/3600

def angular_dist(a1, d1, a2, d2):
  p1 = np.sin(abs(d1-d2)/2)**2
  p2 = np.cos(d1)*np.cos(d2)*np.sin(abs(a1-a2)/2)**2
  p3 = 2*np.arcsin(np.sqrt(p1+p2))
  return np.degrees(p3)

def crossmatch(cat1, cat2, max_dist):
  matches, nomatches = [], []
  cat1id, cat2id = 0, 0
  max_dist_rad = np.radians(max_dist)
  
  start = time.perf_counter()
  
  cat1 = np.radians(cat1)
  cat2 = np.radians(cat2)
  sorted_dec_idxs = np.argsort(cat2[:, 1])
  
  for cat1line in cat1:
    best_match = (0,0,max_dist+1)

    cat2id = 0
    for sorted_idx in sorted_dec_idxs:
      # optimization
      if(cat2[sorted_idx][1] > max_dist_rad + cat1line[1]):
        break
      
      dist = angular_dist(cat1line[0], cat1line[1], cat2[sorted_idx][0], cat2[sorted_idx][1])
      
      # update best match
      if(dist <= best_match[2]):
        best_match = (cat1id, sorted_idx, dist)
    
    #process best match and outputs
    if(best_match[2] <= max_dist):
      matches.append(best_match)
    else:
      nomatches.append(cat1id)
      
    cat1id += 1
    
  return (matches, nomatches, time.perf_counter() - start)

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('\n Time taken:', time_taken)

matches: [(0, 0, 2.0000000000000027), (2, 2, 1.7420109046547023)]
unmatched: [1]
time taken: 0.00014851301345278345
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

 Time taken: 0.0017332555350528245


<h3>4. Boxing the Binary Search</h3>
We can improve on the previous optimisation further by not only stopping the search once it gets past declination of the object to be matched, but starting the search as close as possible to the object. To summarise, the modification is:

Sort the second catalogue objects by order of declination;
Start the search loop at the first second catalogue object with declination greater than [ϴ -r] ;
Finish the search loop at the last second catalogue object with declination less than [ϴ + r].
Boxing in the search in this way saves on calculating the distances for almost the entire second catalogue.

We just need to find a fast way to find the second catalogue objects nearest to the boundaries of [(ϴ -r), (ϴ +r)] so we know where to start and finish our search.

If a list is sorted, it can be much faster to find the index of some element using a binary search, rather than doing comparisons on every element in the list.

A binary search splits the list in half repeatedly, continuing the search in the half that may contain the target element.

This seems a roundabout way of finding 15 in a list of 10 to 19, but note that only 3 comparisons were made, whereas 6 comparisons would been made if we'd just searched the whole list.

On big arrays the savings can be enormous. Whereas 500 comparisons are on average necessary to find an element in a list of length 1000 with direct searching, only 10 are necessary with a binary search.

In [8]:
import numpy as np
import statistics
import time

def hms2dec(h, m, s):
  return 15*(h + m/60 + s/3600)

def dms2dec(d, m, s):
  if (d >= 0):
    return d + m/60 + s/3600
  else:
    return d - m/60 - s/3600

def angular_dist(a1, d1, a2, d2):
  p1 = np.sin(abs(d1-d2)/2)**2
  p2 = np.cos(d1)*np.cos(d2)*np.sin(abs(a1-a2)/2)**2
  return 2*np.arcsin(np.sqrt(p1+p2))

def crossmatch(cat1, cat2, max_dist):
  matches, nomatches = [], []
  cat1id, cat2id = 0, 0
  
  start = time.perf_counter()
  
  max_dist = np.radians(max_dist)
  cat1 = np.radians(cat1)
  cat2 = np.radians(cat2)
  decs = cat2[:, 1] #extract the array of the 2º column, the declinations "y"
  sorted_dec_idxs = np.argsort(decs) #returns an array of sorted indexes
  dec_sorted = decs[sorted_dec_idxs] #orders the array so that it's sorted
  
  for cat1line in cat1:
    best_match = (0,0,max_dist+1)

    min_index = np.searchsorted(dec_sorted, cat1line[1] - max_dist, side='left')
    max_index = np.searchsorted(dec_sorted, cat1line[1] + max_dist, side='left')
        
    for sorted_idx in range(min_index,max_index):
      #the range iterates over dec_sorted, i.e., sorted array
      dist = angular_dist(cat1line[0], cat1line[1], cat2[sorted_dec_idxs[sorted_idx]][0], cat2[sorted_dec_idxs[sorted_idx]][1])
      
      # update best match
      if(dist <= best_match[2]):
        best_match = (cat1id, sorted_dec_idxs[sorted_idx], dist)
    
    #process best match and outputs
    if(best_match[2] <= max_dist):
      matches.append(best_match)
    else:
      nomatches.append(cat1id)
      
    cat1id += 1
    
  return (matches, nomatches, time.perf_counter() - start)



# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('\n Time taken:', time_taken)

matches: [(0, 0, 0.03490658503988664), (2, 2, 0.03040382589186957)]
unmatched: [1]
time taken: 0.00015364456066890853
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

 Time taken: 0.00030064829553566597


<h2>Crossmatching in Astropy using kd trees</h2>

Crossmatching is a very common task in astrophysics, so it's natural that it's had optimised implementations written of it already. 
A popular implementation in Python is found in the Astropy module and it uses objects called k-d trees to perform crossmatching incredibly quickly.

Astropy constructs a k-d tree out of the second catalogue, letting it search through for a match for each object in the first catalogue efficiently. Constructing a k-d tree is similar to the binary search you saw earlier. The k-dimensional space is divided into two parts recursively until each division only contains only a single object. Creating a k-d tree from an astronomy catalogue works like this:

Find the object with the median right ascension, split the catalogue into objects left and right partitions of this
Find the objects with the median declination in each partition, split the partitions into smaller partitions of objects down and up of these
Find the objects with median right ascension in each of the partitions, split the partitions into smaller partitions of objects left and right of these
Repeat 2-3 until each partition only has one object in it
This creates a binary tree where each object used to split a partition (a node) links to the two objects that then split the partitions it has created (its children).
![title](k-d_tree_1.png)

  Since each node branches into two children, a catalogue of objects will have, on average <b>log2(N)</b> nodes from the root to any leaf. So while it seems like a lot of effort to create a k-d tree, doing so lets you, for example, search the entire SuperCOSMOS catalogue of 250 million objects using <b> only 28 distance calculations.</b>
  

The SkyCoord objects are general purpose sky catalogue storage and manipulation objects in Astropy. They take anything that looks like an array of coordinates as long as you specify the units (here we specify degrees with u.degree) and a reference frame


In [7]:
import numpy as np
import statistics
import time
from astropy.coordinates import SkyCoord
from astropy import units as u


def crossmatch(cat1, cat2, max_dist):
    matches = []
    nomatches = []

    start = time.perf_counter()

    skycat1 = SkyCoord(cat1 * u.degree, frame='icrs')
    skycat2 = SkyCoord(cat2 * u.degree, frame='icrs')
    closest_ids, closest_dists, closest_dists3d = skycat1.match_to_catalog_sky(skycat2)
    closest_dists_deg = closest_dists.value

    for cat1idx in range(len(cat1)):
        if closest_dists_deg[cat1idx] > max_dist:
            nomatches.append(cat1idx)
        else:
            matches.append((cat1idx, closest_ids[cat1idx], closest_dists_deg[cat1idx]))

    return (matches, nomatches, time.perf_counter() - start)


# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
    # The example in the question
    cat1 = np.array([[180, 30], [45, 10], [300, -45]])
    cat2 = np.array([[180, 32], [55, 10], [302, -44]])
    matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
    print('matches:', matches)
    print('unmatched:', no_matches)
    print('time taken:', time_taken)


    # A function to create a random catalogue of size n
    def create_cat(n):
        ras = np.random.uniform(0, 360, size=(n, 1))
        decs = np.random.uniform(-90, 90, size=(n, 1))
        return np.hstack((ras, decs))


    # Test your function on random inputs
    np.random.seed(0)
    cat1 = create_cat(10)
    cat2 = create_cat(20)
    matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
    print('matches:', matches)
    print('unmatched:', no_matches)
    print('Time taken:', time_taken)

matches: [(0, 0, 2.0000000000000036), (2, 2, 1.7420109046547163)]
unmatched: [1]
time taken: 0.09569762026006856
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Time taken: 0.09845114812333122
